# Descripción

Este notebook contiene los pasos requeridos para asignar las combinaciones, previamente asignadas, profesor-curso a un salón de acuerdo a los requerimientos institucionales.

In [68]:
import json

# READ DATA

## first stage model output

In [69]:
# basic parameters
DAY_LENGTH=16

# Horizonte de planificación
HORIZON=5

#  Días de la semana
DAYS=['Lunes','Martes','Miercoles','Jueves','Viernes']


In [70]:
# Este diccionario contiene un mapa de los slots de tiempo a los días y horas

actual_time={}
for d in range(HORIZON):
    for h in range(DAY_LENGTH):
        actual_time[d*16+h]={"slot":DAYS[d]+' '+str(h+6)+':00', "day":d, "hour":h,'slot_abr':DAYS[d][0]+str(h)}

In [71]:
# read model_output.json
allocations=json.load(open('data/model_output.json','r'))
#allocations

In [72]:
groups=allocations['courses']
teachers=allocations['teachers']
pairs=allocations['ensembles']
teachers=set([int(t) for t in teachers])
groups=set([g for g in groups])
pairs=[(int(p[0]) ,p[1]) for p in pairs]

In [73]:
courses_by_teacher={}
for p in pairs:
    if p[0] not in courses_by_teacher:
        courses_by_teacher[p[0]]=[p[1]]
    else:
        courses_by_teacher[p[0]].append(p[1])

# teachers by course
teachers_by_course={}
for p in pairs:
   teachers_by_course[p[1]]=p[0]




## Initial data load

In [74]:
# read data.json
horas_curso=json.load(open('data/horas_curso.json','r'))
grupo_requerimiento=json.load(open('data/grupo_requerimiento.json','r'))
ubicacion_semestral=json.load(open('data/ubicacion_semestral.json','r'))
disponibilidad_docentes=json.load(open('data/disponibilidad.json','r'))

In [75]:
tipo_salon={'A','B','C','D','E','F','G'}
hora_disponibilidad_x_salon={
    'A':10
    ,'B':0
    ,'C':0
    ,'D':10
    ,'E':10
    ,'F':10
    ,'G':10
}

cantidad_de_salones={
 'A':11
 ,'B':9
 ,'C':3
 ,'D':1
 ,'E':1
 ,'F':2
 ,'G':1   
}

salas_de_inform={"C","D"}

clases_sala_infor={
    "2027-0059",
    "9937-0012",
    "6612-0001",
    "8830-0008",
    "8837-0021",
    "8830-0006",
    "9937-0013",
    "6611-0088",
    "8827-0008",
    "8830-0009",
    "6069-0018",
    "6069-0127",
    "9937-0014",
    "8821-0010",
    "9937-0018"
    
}



# Modelo

In [76]:
I=set(teachers)
J=set(groups)
K=set(pairs)
C= tipo_salon

In [77]:
from itertools import product

# courses that are the same
same_course={}
# courses that are taught by the same teacher
same_teacher={}

# delta semestres
delta_semestre={}
for j1,j2 in product(J,J):
    same_course[(j1,j2)]=0
    same_teacher[(j1,j2)]=0
    if j1[:9]==j2[:9]:
        same_course[(j1,j2)]=1
    delta_semestre[(j1,j2)]=abs(ubicacion_semestral[j1[:9]]["Semestre"]-ubicacion_semestral[j2[:9]]["Semestre"])

for p in pairs:
    for p2 in pairs:
        if p[0]==p2[0]:
            same_teacher[(p[1],p2[1])]=1

# course and time slot compatibility
time_compatibility={}
for c,j in product(C,J):
    for s in actual_time.keys():
        time_compatibility[(c,j,s)]=0
        if actual_time[s]['hour'] + horas_curso[j[:9]]['horas']-1<DAY_LENGTH:
            time_compatibility[(c,j,s)]=1

print(disponibilidad_docentes)
valid_keys=[]
for c,j,s in product(C,J,actual_time.keys()):
    if time_compatibility[(c,j,s)]==1 and s in disponibilidad_docentes[str(teachers_by_course[j])]:
        valid_keys.append((c,j,s))



{'326151': [3, 4, 5, 6, 8, 9, 10, 11, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 36, 37, 38, 39, 40, 43, 44, 60, 61, 62, 63, 64, 65, 66, 76, 77, 78, 79, 80, 81, 82, 83], '336392': [3, 4, 5, 8, 9, 10, 11, 12, 13, 14, 25, 26, 27, 28, 29, 30, 31, 32, 38, 39, 43, 44, 45, 46, 47, 48, 49, 54, 55, 56, 59, 60, 61, 62, 63, 79, 80, 81, 82, 83], '183645': [2, 3, 4, 5, 6, 9, 10, 11, 12, 25, 26, 27, 28, 29, 30, 31, 32, 43, 44, 45, 46, 47, 48, 49, 55, 56, 59, 60, 61, 62, 77, 78, 79, 80, 81, 82, 87, 88, 89, 90], '274691': [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 21, 22, 23, 28, 29, 30, 31, 32, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83], '259554': [7, 8, 9, 10, 11, 12, 13, 14, 15, 22, 23, 29, 30, 31, 32, 42, 43, 44, 45, 46, 47, 48, 49, 61, 62, 63, 64, 65, 66, 79, 80, 81, 82, 83, 87, 88, 89, 90, 91, 92], '331121': [7, 8, 9, 10, 11, 12, 13, 14, 15, 29, 30, 31, 32, 42, 43, 44, 45, 46, 59, 60, 61, 62, 63, 64, 75, 76, 77, 78, 79, 80, 81, 82, 83, 87, 88, 89, 90, 91, 92]

In [18]:
import gurobipy as gp

# declare an empty model
m=gp.Model('Asignacion de salones')
m.setParam('OutputFlag',False)

# model sense
m.modelSense=gp.GRB.MAXIMIZE

# declares variables

# y representa si el cursu j se asigna a un salón tipo c iniciando en el slot s
y=m.addVars(valid_keys,vtype=gp.GRB.BINARY,name='y')
# x representa si el curso j ocupa un salón tipo c durante el slot s
x=m.addVars([(c,j,s) for c in C for j in J for s in actual_time.keys()],vtype=gp.GRB.BINARY,name='x')
# indica si dos cursos se solapan en algún momento
z=m.addVars([(j1,j2) for j1,j2 in product(J,J) if j1!=j2],vtype=gp.GRB.BINARY,name='z')

Set parameter Username


Academic license - for non-commercial use only - expires 2024-04-04


El objetivo es generar una programación tal que los cursos de semestres similares se
programan en diferentes momentos del día. Es decir, queremos maximizar la diferencia de semestre entre los cursos que se deban programar a la misma hora.

$ \max:\sum_{{j1,j2 \in J, j1 \neq j2}} z[j1,j2] \cdot \delta_{\text{{semestre}}}[j1,j2]$


In [19]:
# funcion objetivo:
# maximizar la suma de los deltas semestrales de los cursos que se imparten
m.setObjective(gp.quicksum(z[j1,j2]*delta_semestre[j1,j2] for j1,j2 in product(J,J) if j1!=j2))

Restricción 1: Todos los cursos deben tener exactamente una asignación de inicio.

$\sum_{c,j1,s \in \text{{valid\_keys}}, j1=j} y[(c,j1,s)] = 1 \quad \forall j \in J$


In [20]:
# each course starts at one time slot
m.addConstrs((gp.quicksum(y[(c,j1,s)] for c,j1,s in valid_keys if j1==j)==1 for j in J),name='each_course_starts_at_one_time_slot')
0

0

Restricción 2:

si un curso da inicio durante el slot s en el salón tipo c, y tiene una duración t, ese curso debe estar ocupando un slot entre s, s+1 hasta s+t-1

$  
\sum_{s1 \in \text{{range}}(s,s+\text{{horas\_curso}}[j1[:9]]['horas']) \atop (c,j1,s1) \in \text{{valid\_keys}}} x[(c,j1,s1)] \geq y[(c,j1,s)] \cdot \text{{horas\_curso}}[j1[:9]]['horas'] \forall c, j1, s \in \text{{valid\_keys}}
$

In [22]:
"""cont=0
for c,j1,s in valid_keys:
    lhs=gp.quicksum(x[(c,j1,s1)] for s1 in range(s,s+horas_curso[j1[:9]]['horas']) )
    rhs=y[(c,j1,s)]*horas_curso[j1[:9]]['horas']
    print(lhs,">=",rhs)
    cont+=1
    if cont==16:
        break"""
    

m.addConstrs((gp.quicksum(x[(c,j1,s1)] for s1 in range(s,s+horas_curso[j1[:9]]['horas'])  )
             >=y[(c,j1,s)]*horas_curso[j1[:9]]['horas'] for c,j1,s in valid_keys))
0

0

Restricción 3:
Cada curso es asignado un bloque de tiempo equivalente a las horas que requiere


$ \sum_{c,j1,s \in \text{{valid\_keys}} \atop j1=j} x[(c,j1,s)] = \text{{horas\_curso}}[j[:9]]['horas'] \forall j \in J$



In [23]:
m.addConstrs((gp.quicksum(x[(c,j1,s)] for c,j1,s in valid_keys if j1==j)==horas_curso[j[:9]]['horas'] for j in J),name='each_course_is_assigned_the_time_it_requires')
0

0

$\forall j1, j2 \in J \quad \text{{s.t.}} \quad j1 \neq j2, \forall c1, c2 \in C, \forall s \in \text{{actual\_time.keys()}}, z[j1,j2] \geq x[c1,j1,s] + x[c2,j2,s] - 1$


In [24]:
m.addConstrs((z[j1,j2]>=x[c1,j1,s]+x[c2,j2,s]-1 for j1,j2 in product(J,J) if j1!=j2 for c1 ,c2 in product(C,C) for s in actual_time.keys()),name='si_dos_cursos_se_imparten_a_la_misma_hora_en_cualquier_salon_se_debe_activar_la_variable_z_que_lleva_cuentas_de_los_solapamientos')
0

0

In [25]:

m.addConstrs((gp.quicksum(x[(c,j,s)] for j in J )<=cantidad_de_salones[c]  for c in C for s in actual_time.keys()),name='cada_curso_se_imparte_en_un_maximo_de_un_salon_de_cada_tipo')
0


0

In [26]:
# si el salón requiere sala de sistemas, se debe programar en una de las salas de sistemas {"C","D"}

m.addConstrs((gp.quicksum(y[(c,j,s)] for c in salas_de_inform for s in actual_time.keys() if (c,j,s) in valid_keys)==1 for j in J if j[:9] in clases_sala_infor ),name='si_el_curso_requiere_sala_de_sistemas_se_debe_programar_en_una_de_las_salas_de_sistemas')
m.addConstrs((gp.quicksum(y[(c,j,s)] for c in salas_de_inform for s in actual_time.keys() if (c,j,s) in valid_keys)==0 for j in J if not( j[:9] in clases_sala_infor) ),name='si_el_curso_requiere_sala_de_sistemas_se_debe_programar_en_una_de_las_salas_de_sistemas')
0

0

In [27]:
# una clase no puede empezar en un slot si no está aún disponible el salón
m.addConstrs((y[(c,j,s)]==0 for c,j,s in valid_keys if actual_time[s]['hour']<hora_disponibilidad_x_salon[c]),name='una_clase_no_puede_empezar_en_un_slot_si_no_esta_aun_disponible_el_salon')

{('F', '8830-0004-0', 3): <gurobi.Constr *Awaiting Model Update*>,
 ('F', '8830-0004-0', 4): <gurobi.Constr *Awaiting Model Update*>,
 ('F', '8830-0004-0', 5): <gurobi.Constr *Awaiting Model Update*>,
 ('F', '8830-0004-0', 6): <gurobi.Constr *Awaiting Model Update*>,
 ('F', '8830-0004-0', 8): <gurobi.Constr *Awaiting Model Update*>,
 ('F', '8830-0004-0', 9): <gurobi.Constr *Awaiting Model Update*>,
 ('F', '8830-0004-0', 19): <gurobi.Constr *Awaiting Model Update*>,
 ('F', '8830-0004-0', 20): <gurobi.Constr *Awaiting Model Update*>,
 ('F', '8830-0004-0', 21): <gurobi.Constr *Awaiting Model Update*>,
 ('F', '8830-0004-0', 22): <gurobi.Constr *Awaiting Model Update*>,
 ('F', '8830-0004-0', 23): <gurobi.Constr *Awaiting Model Update*>,
 ('F', '8830-0004-0', 25): <gurobi.Constr *Awaiting Model Update*>,
 ('F', '8830-0004-0', 36): <gurobi.Constr *Awaiting Model Update*>,
 ('F', '8830-0004-0', 37): <gurobi.Constr *Awaiting Model Update*>,
 ('F', '8830-0004-0', 38): <gurobi.Constr *Awaiting Mo

In [29]:
time_limit=60
m.setParam('TimeLimit',time_limit)
m.optimize()

# print model status
if m.status==gp.GRB.OPTIMAL:
    print('Optimal solution found with objective: %g' % m.objVal)
elif m.status==gp.GRB.TIME_LIMIT:
    print('Time limit reached with objective: %g' % m.objVal)
else:
    print('Model is infeasible')

Model is infeasible


In [56]:
allocations

{'courses': ['9937-0021-0',
  '9939-0018-0',
  '8831-0028-0',
  '5569-0011-2',
  '9935-0033-0',
  '8821-0010-0',
  '6069-0127-0',
  '5569-0011-1',
  '5569-0011-0',
  '6611-0004-1',
  '6611-0004-0',
  '8830-0008-0',
  '9937-0018-0',
  '8830-0009-0',
  '9937-0014-0',
  '9937-0062-1',
  '6069-0011-0',
  '9937-0073-0',
  '8828-0028-0',
  '9935-0032-0',
  '9935-0012-0',
  '9935-0031-0',
  '9937-0062-0',
  '8827-0008-0',
  '6069-0012-0',
  '8827-0120-0',
  '9937-0063-0',
  'CEEM-0014-0',
  '8830-0010-0',
  '6612-0001-0',
  '6069-0175-0',
  '8833-0083-0',
  '5555-0010-0',
  '6069-2060-0',
  '6069-0018-0',
  '6069-0014-0',
  '8830-0012-0',
  '8830-0005-0',
  '8821-0002-0',
  '5566-0009-0',
  '8833-0083-0',
  '6611-0088-0',
  '6069-0078-0',
  '8830-0011-0',
  '9937-0072-0',
  '9937-0011-0',
  '9937-0012-0',
  '9937-0074-0',
  '9937-0076-0',
  '2027-0059-1',
  '2027-0059-0',
  '5559-0014-0',
  '8830-0001-0',
  '8830-0007-0',
  '8830-0004-0',
  '8830-0006-0',
  '8833-0021-0',
  '9937-0013-0',
  '

In [54]:
print('Time elapsed: %g' % m.Runtime)
# print the keys for the y variables that are equal to 1
for c,j,s in valid_keys:
    
    if y[c,j,s].x>=0.9:
        print("la clase",horas_curso[j[:9]]['ASIGNATURA'],"se imparte en un salón tipo ",c,"en el horario del ",actual_time[s]["slot"])

Time elapsed: 7.57121
la clase Planeación_Control_Producción se imparte en un salón tipo  D en el horario del  Lunes 18:00
la clase Diseño_Experimental se imparte en un salón tipo  D en el horario del  Martes 16:00
la clase Procesos_Industriales se imparte en un salón tipo  G en el horario del  Martes 17:00
la clase Gestión_Tecnología_Innovación se imparte en un salón tipo  G en el horario del  Lunes 18:00
la clase Metodología_Investigación se imparte en un salón tipo  A en el horario del  Viernes 19:00
la clase Diseño_Mantenimiento_Plantas   se imparte en un salón tipo  A en el horario del  Miercoles 17:00
la clase OCI-I_Gerencia_contemporanea se imparte en un salón tipo  A en el horario del  Miercoles 17:00
la clase Economía se imparte en un salón tipo  A en el horario del  Miercoles 18:00
la clase Algebra_Lineal se imparte en un salón tipo  A en el horario del  Lunes 19:00
la clase Cálculo_Vectorial se imparte en un salón tipo  A en el horario del  Martes 18:00
la clase Legislación 